In [60]:
pip install pyomo

Note: you may need to restart the kernel to use updated packages.


In [61]:
# from collections import defaultdict
from pyomo.util.infeasible import log_infeasible_constraints

from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np
import datetime

In [62]:
pd.set_option('display.max_columns', 20)

In [63]:
def remove_nan(dic):  # remove nan from a dictionary
    for k in dic.keys():
        for v in range(len(dic[k])):
            try:
                if np.isnan(dic[k][v]):
                    del dic[k][v]
            except:
                pass
    return dic


## Reading Input Data

### Employee Requirement For Assignment

#### Read Day1 data

In [64]:
# Employee Requirement for Assignment in each TSlot
TimeAssEmpReq1 = pd.read_csv('/home/ZONE24X7-CMB/cheranil/Documents/Kronos/Data/Weekly/Day1/Time_Ass_Req_Schedule.csv', index_col=False)

# #TimeAssEmpReq['Time'] = TimeAssEmpReq['Time'].str.split(' ').str[0].replace(':', '.', regex=True).str.strip("0")
TimeAssEmpReq1['Time'] = TimeAssEmpReq1['Time'].str.split(' ').str[0]
# #timeDF=(pd.to_timedelta(TimeAssEmpReq['Time'].str.strip()))
TimeAssEmpReq1['Time']=(pd.to_datetime(TimeAssEmpReq1['Time'].str.strip(), format='%H:%M:%S')).dt.time
TimeAssEmpReq1.head()

,Day,Time,Assignment,Requirement,Scheduled
0,15/11/2018,09:00:00,Downtown Lobby,3.8,2
1,15/11/2018,09:15:00,Downtown Lobby,3.2,2
2,15/11/2018,09:30:00,Downtown Lobby,2.6,2
3,15/11/2018,09:45:00,Downtown Lobby,2.8,2
4,15/11/2018,10:00:00,Downtown Lobby,4.1,2


#### Read Day2 data

In [65]:
# Employee Requirement for Assignment in each TSlot
TimeAssEmpReq2 = pd.read_csv('/home/ZONE24X7-CMB/cheranil/Documents/Kronos/Data/Weekly/Day2/Time_Ass_Req_Schedule2.csv', index_col=False)

# #TimeAssEmpReq['Time'] = TimeAssEmpReq['Time'].str.split(' ').str[0].replace(':', '.', regex=True).str.strip("0")
TimeAssEmpReq2['Time'] = TimeAssEmpReq2['Time'].str.split(' ').str[0]
# #timeDF=(pd.to_timedelta(TimeAssEmpReq['Time'].str.strip()))
TimeAssEmpReq2['Time']=(pd.to_datetime(TimeAssEmpReq2['Time'].str.strip(), format='%H:%M:%S')).dt.time

TimeAssEmpReq2.head()


,Day,Time,Assignment,Requirement,Scheduled
0,16/11/2018,09:00:00,Downtown Lobby,7.4,3
1,16/11/2018,09:15:00,Downtown Lobby,3.7,3
2,16/11/2018,09:30:00,Downtown Lobby,4.0,3
3,16/11/2018,09:45:00,Downtown Lobby,3.7,3
4,16/11/2018,10:00:00,Downtown Lobby,3.9,3


In [66]:
TimeAssEmpReq = pd.concat([TimeAssEmpReq1, TimeAssEmpReq2])
TimeAssEmpReq.head()

,Day,Time,Assignment,Requirement,Scheduled
0,15/11/2018,09:00:00,Downtown Lobby,3.8,2
1,15/11/2018,09:15:00,Downtown Lobby,3.2,2
2,15/11/2018,09:30:00,Downtown Lobby,2.6,2
3,15/11/2018,09:45:00,Downtown Lobby,2.8,2
4,15/11/2018,10:00:00,Downtown Lobby,4.1,2


### Employee Eligibility Details

#### Day1

In [67]:
# define employees
EmpData1 = pd.read_csv('/home/ZONE24X7-CMB/cheranil/Documents/Kronos/Data/Weekly/Day1/EmpTasks.csv', index_col=False)
EmpData1.head()

,Day,Employee,Task
0,15/11/2018,Employee501603,Downtown Platform
1,15/11/2018,Employee501603,MSR Opener Platform
2,15/11/2018,Employee501603,MSR Closer Platform
3,15/11/2018,Employee501715,Downtown Lobby
4,15/11/2018,Employee501715,MSR Opener Lobby


#### Day2

In [68]:
EmpData2 = pd.read_csv('/home/ZONE24X7-CMB/cheranil/Documents/Kronos/Data/Weekly/Day2/EmpTasks2.csv', index_col=False)
EmpData2.head()

,Day,Employee,Task
0,16/11/2018,Employee501603,Downtown Platform
1,16/11/2018,Employee501603,MSR Opener Platform
2,16/11/2018,Employee501603,MSR Closer Platform
3,16/11/2018,Employee501715,Downtown Lobby
4,16/11/2018,Employee501715,MSR Opener Lobby


In [69]:
EmpData = pd.concat([EmpData1, EmpData2])
EmpData.head()

,Day,Employee,Task
0,15/11/2018,Employee501603,Downtown Platform
1,15/11/2018,Employee501603,MSR Opener Platform
2,15/11/2018,Employee501603,MSR Closer Platform
3,15/11/2018,Employee501715,Downtown Lobby
4,15/11/2018,Employee501715,MSR Opener Lobby


### Input Data Dictionaries

In [70]:
# define Days
Days = TimeAssEmpReq.Day.unique()
print(Days)

['15/11/2018' '16/11/2018']


In [71]:
# define Time Slots
TSlots = TimeAssEmpReq.Time.unique()
print(TSlots)

[datetime.time(9, 0) datetime.time(9, 15) datetime.time(9, 30)
 datetime.time(9, 45) datetime.time(10, 0) datetime.time(10, 15)
 datetime.time(10, 30) datetime.time(10, 45) datetime.time(11, 0)
 datetime.time(11, 15) datetime.time(11, 30) datetime.time(11, 45)
 datetime.time(12, 0) datetime.time(12, 15) datetime.time(12, 30)
 datetime.time(12, 45) datetime.time(1, 0) datetime.time(1, 15)
 datetime.time(1, 30) datetime.time(1, 45) datetime.time(2, 0)
 datetime.time(2, 15) datetime.time(2, 30) datetime.time(2, 45)
 datetime.time(3, 0) datetime.time(3, 15) datetime.time(3, 30)
 datetime.time(3, 45) datetime.time(4, 0) datetime.time(4, 15)
 datetime.time(4, 30) datetime.time(4, 45) datetime.time(8, 30)
 datetime.time(8, 45) datetime.time(5, 0) datetime.time(5, 15)]


In [72]:
Employees = EmpData.Employee.unique()
print(Employees)

['Employee501603' 'Employee501715' 'Employee501716' 'Employee501721'
 'Employee501768' 'Employee501792' 'Employee501587' 'Employee501595']


In [73]:
# Assignments
Assignments = TimeAssEmpReq.Assignment.unique()
Assignments

array(['Downtown Lobby', 'Downtown Platform', 'Teller Opener Platform',
       'MSR Opener Platform', 'Teller Closer Platform',
       'MSR Closer Platform', 'Teller Opener Lobby', 'MSR Opener Lobby',
       'Teller Closer Lobby', 'MSR Closer Lobby', 'BM Admin Time',
       'ABM Admin Time', 'BMT Meeting'], dtype=object)

In [74]:
# Assignments for date and timeslot
AssTslots = TimeAssEmpReq.groupby(['Day','Time'])['Assignment'].apply(list).to_dict()
Ass_Tslots = remove_nan(AssTslots)
print(AssTslots)

{('15/11/2018', datetime.time(1, 0)): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], ('15/11/2018', datetime.time(1, 15)): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], ('15/11/2018', datetime.time(1, 30)): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], ('15/11/2018', datetime.time(1, 45)): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], ('15/11/2018', datetime.time(2, 0)): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], ('15/11/2018', datetime.time(2, 15)): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], ('15/11/2018', datetime.time(2, 30)): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], ('15/11/2018', datetime.time(2, 45)): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], ('15/11/2018', datetime.time(3, 0)): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Ti

In [75]:
# Employee Eligibility for each assignments
Emp_Elig = EmpData.groupby(['Day','Employee'])['Task'].apply(list).to_dict()
Emp_Elig = remove_nan(Emp_Elig)
print(Emp_Elig)


{('15/11/2018', 'Employee501587'): ['Downtown Platform', 'Downtown Lobby'], ('15/11/2018', 'Employee501595'): ['Downtown Platform', 'MSR Opener Platform', 'MSR Closer Platform'], ('15/11/2018', 'Employee501603'): ['Downtown Platform', 'MSR Opener Platform', 'MSR Closer Platform'], ('15/11/2018', 'Employee501715'): ['Downtown Lobby', 'MSR Opener Lobby', 'MSR Closer Lobby'], ('15/11/2018', 'Employee501716'): ['Downtown Lobby', 'Teller Closer Lobby'], ('15/11/2018', 'Employee501721'): ['Downtown Platform', 'MSR Opener Platform', 'MSR Closer Platform', 'BM Admin Time'], ('15/11/2018', 'Employee501768'): ['Downtown Lobby', 'Teller Closer Lobby'], ('15/11/2018', 'Employee501792'): ['Downtown Platform', 'MSR Opener Platform', 'MSR Closer Platform'], ('16/11/2018', 'Employee501587'): ['Downtown Platform', 'Downtown Lobby'], ('16/11/2018', 'Employee501595'): ['Downtown Platform', 'MSR Opener Platform', 'MSR Closer Platform'], ('16/11/2018', 'Employee501603'): ['Downtown Platform', 'MSR Opener P

### Employee Min Max Hours

#### Day1

In [76]:
EmpWorkHours1 = pd.read_csv('/home/ZONE24X7-CMB/cheranil/Documents/Kronos/Data/Weekly/Day1/EmpWorkHours.csv', index_col=False)
EmpWorkHours1.head()

,Day,Employee,Min_Work,Max_Work
0,15/11/2018,Employee501603,4,8
1,15/11/2018,Employee501715,4,8
2,15/11/2018,Employee501716,4,8
3,15/11/2018,Employee501721,4,8
4,15/11/2018,Employee501768,4,8


#### Day2

In [77]:
EmpWorkHours2 = pd.read_csv('/home/ZONE24X7-CMB/cheranil/Documents/Kronos/Data/Weekly/Day2/EmpWorkHours2.csv', index_col=False)
EmpWorkHours2.head()

,Day,Employee,Min_Work,Max_Work
0,16/11/2018,Employee501603,6,8
1,16/11/2018,Employee501715,6,8
2,16/11/2018,Employee501716,6,8
3,16/11/2018,Employee501721,6,8
4,16/11/2018,Employee501768,6,8


In [78]:
EmpWorkHours = pd.concat([EmpWorkHours1,EmpWorkHours2])
EmpWorkHours.head()

,Day,Employee,Min_Work,Max_Work
0,15/11/2018,Employee501603,4,8
1,15/11/2018,Employee501715,4,8
2,15/11/2018,Employee501716,4,8
3,15/11/2018,Employee501721,4,8
4,15/11/2018,Employee501768,4,8


In [79]:
#Dictionaries of Min Max hours
EmpMinWork = pd.Series(EmpWorkHours.Min_Work.values, index=[EmpWorkHours.Day,EmpWorkHours.Employee]).to_dict()
EmpMaxWork = pd.Series(EmpWorkHours.Max_Work.values, index=[EmpWorkHours.Day,EmpWorkHours.Employee]).to_dict()
print(EmpMaxWork)

{('15/11/2018', 'Employee501603'): 8, ('15/11/2018', 'Employee501715'): 8, ('15/11/2018', 'Employee501716'): 8, ('15/11/2018', 'Employee501721'): 8, ('15/11/2018', 'Employee501768'): 8, ('15/11/2018', 'Employee501792'): 8, ('15/11/2018', 'Employee501587'): 8, ('15/11/2018', 'Employee501595'): 8, ('16/11/2018', 'Employee501603'): 8, ('16/11/2018', 'Employee501715'): 8, ('16/11/2018', 'Employee501716'): 8, ('16/11/2018', 'Employee501721'): 8, ('16/11/2018', 'Employee501768'): 8, ('16/11/2018', 'Employee501792'): 8, ('16/11/2018', 'Employee501587'): 9, ('16/11/2018', 'Employee501595'): 8}


#### Weekly Min Max hours and days

In [80]:
EmpWeekly = pd.read_csv('/home/ZONE24X7-CMB/cheranil/Documents/Kronos/Data/Weekly/EmpWeeklyHours.csv', index_col=False)
EmpWeekly.head()

,Employee,Min_Hours,Max_Hours,Max_days
0,Employee501603,8,16,1
1,Employee501715,8,16,1
2,Employee501716,8,16,1
3,Employee501721,8,16,1
4,Employee501768,12,16,2


In [81]:
EmpWeeklyMinHours = pd.Series(EmpWeekly.Min_Hours.values, index=EmpWeekly.Employee).to_dict()
EmpWeeklyMaxHours = pd.Series(EmpWeekly.Max_Hours.values, index=EmpWeekly.Employee).to_dict()
EmpWeeklyMaxDays = pd.Series(EmpWeekly.Max_days.values, index=EmpWeekly.Employee).to_dict()
print(EmpWeeklyMaxHours)

{'Employee501603': 16, 'Employee501715': 16, 'Employee501716': 16, 'Employee501721': 16, 'Employee501768': 16, 'Employee501792': 16, 'Employee501587': 16, 'Employee501595': 16}


#### Pinned Task data

In [82]:
PinnedTasks = pd.read_csv('/home/ZONE24X7-CMB/cheranil/Documents/Kronos/Data/Weekly/PinnedTasks.csv', index_col=False)
PinnedTasks['Time'] = PinnedTasks['Time'].str.split(' ').str[0]
# #timeDF=(pd.to_timedelta(TimeAssEmpReq['Time'].str.strip()))
PinnedTasks['Time']=(pd.to_datetime(PinnedTasks['Time'].str.strip(), format='%H:%M:%S')).dt.time

PinnedTasks.head()

,Employee,Day,Time,Task
0,Employee501603,15/11/2018,09:00:00,Downtown Platform
1,Employee501603,15/11/2018,09:15:00,Downtown Platform
2,Employee501603,15/11/2018,09:30:00,Downtown Platform
3,Employee501603,15/11/2018,09:45:00,Downtown Platform
4,Employee501715,16/11/2018,09:00:00,Downtown Lobby


In [83]:
PinnedTaskList = [tuple(x) for x in PinnedTasks.to_numpy()]
PinnedTaskList

[('Employee501603', '15/11/2018', datetime.time(9, 0), 'Downtown Platform'),
 ('Employee501603', '15/11/2018', datetime.time(9, 15), 'Downtown Platform'),
 ('Employee501603', '15/11/2018', datetime.time(9, 30), 'Downtown Platform'),
 ('Employee501603', '15/11/2018', datetime.time(9, 45), 'Downtown Platform'),
 ('Employee501715', '16/11/2018', datetime.time(9, 0), 'Downtown Lobby'),
 ('Employee501715', '16/11/2018', datetime.time(9, 15), 'Downtown Lobby'),
 ('Employee501715', '16/11/2018', datetime.time(9, 30), 'Downtown Lobby'),
 ('Employee501715', '16/11/2018', datetime.time(9, 45), 'Downtown Lobby')]

#### Employee Requirement per Assignment and Timeslot

In [84]:
TimeAssEmpReq.head()

,Day,Time,Assignment,Requirement,Scheduled
0,15/11/2018,09:00:00,Downtown Lobby,3.8,2
1,15/11/2018,09:15:00,Downtown Lobby,3.2,2
2,15/11/2018,09:30:00,Downtown Lobby,2.6,2
3,15/11/2018,09:45:00,Downtown Lobby,2.8,2
4,15/11/2018,10:00:00,Downtown Lobby,4.1,2


In [85]:
# Employee Requirement
Emp_Req = TimeAssEmpReq.groupby(['Day','Time'])[['Assignment', 'Requirement']] \
    .apply(lambda x: pd.Series(x.Requirement.values, index=x.Assignment).to_dict()).to_dict()

print(Emp_Req)

{('15/11/2018', datetime.time(1, 0)): {'Downtown Lobby': 3.9, 'Downtown Platform': 3.5, 'BM Admin Time': 1.0, 'ABM Admin Time': 1.0}, ('15/11/2018', datetime.time(1, 15)): {'Downtown Lobby': 5.6, 'Downtown Platform': 3.0, 'BM Admin Time': 1.0, 'ABM Admin Time': 1.0}, ('15/11/2018', datetime.time(1, 30)): {'Downtown Lobby': 5.3, 'Downtown Platform': 2.9, 'BM Admin Time': 1.0, 'ABM Admin Time': 1.0}, ('15/11/2018', datetime.time(1, 45)): {'Downtown Lobby': 4.8, 'Downtown Platform': 2.4, 'BM Admin Time': 1.0, 'ABM Admin Time': 1.0}, ('15/11/2018', datetime.time(2, 0)): {'Downtown Lobby': 3.8, 'Downtown Platform': 2.2, 'BM Admin Time': 1.0, 'ABM Admin Time': 1.0}, ('15/11/2018', datetime.time(2, 15)): {'Downtown Lobby': 4.9, 'Downtown Platform': 2.1, 'BM Admin Time': 1.0, 'ABM Admin Time': 1.0}, ('15/11/2018', datetime.time(2, 30)): {'Downtown Lobby': 4.7, 'Downtown Platform': 2.1, 'BM Admin Time': 1.0, 'ABM Admin Time': 1.0}, ('15/11/2018', datetime.time(2, 45)): {'Downtown Lobby': 4.3, '

In [86]:
# convert Emp_Req dictionaries values in to Lists
l = []
for k in Emp_Req.keys():
    a = list(Emp_Req[k].values())
    l.append(a)

Emp_requiremnt = sum(sum(x) for x in l)  # calculate employee requirement

No_of_Assignments = sum([len(x) for x in Ass_Tslots.values()])

# Model Creation

#### Decision Variables

In [87]:
# Initialize model
model = ConcreteModel()

# variable to represent employee allocation to assignments (decision variable)
model.works = Var(
    (((Employee, Day, TSlot, Assignment) for Employee in Employees for Day in Days for TSlot in TSlots for Assignment in Ass_Tslots[(Day, TSlot)])),
    within=Binary, initialize=0)

model.MI = Var(bounds=(0, 5))
model.MA = Var(bounds=(0, 5))
model.MIW = Var(bounds=(0, 10))

# variable to represent employee allocation to day (decision variable)
model.dayWorks = Var(
    ((Employee, Day) for Employee in Employees for Day in Days),
    within=Binary, initialize=0)

In [88]:
def getPreviousTimeslot(TSLOT):
    return (datetime.datetime.combine(datetime.date(1, 1, 1), TSLOT) - datetime.timedelta(minutes=15)).time()


### Objective Function

In [89]:
# function to define Objective function - original
# def obj_rule(m):
#     obj = (Emp_requiremnt - sum(
#         m.works[Employee, TSlot, Assignment] for Employee in Employees for TSlot in TSlots for Assignment in
#         Ass_Tslots[TSlot]) + m.MI)
#     return obj

In [90]:
# function to define Objective function - Minimize idle times between allocations
def obj_rule(m):
    obj = 3 * (Emp_requiremnt - sum(m.works[Employee, Day, TSlot, Assignment] 
                for Employee in Employees 
                for Day in Days
                for TSlot in TSlots 
                for Assignment in Ass_Tslots[(Day,TSlot)]) #Minimize the gap between allocated and scheduled
          ) 
    + m.MI 
    + m.MIW
    + sum((max((value(m.works[Employee, Day, TSlot, Assignment]) - value(m.works[(Employee, Day, getPreviousTimeslot(TSlot), Assignment)])) , 
               (value(m.works[(Employee, Day, getPreviousTimeslot(TSlot), Assignment)]) - value(m.works[Employee, Day, TSlot, Assignment]) ) )) 
          for Employee in Employees
          for Day in Days
          for TSlot in TSlots[1:] 
          for Assignment in Ass_Tslots[(Day,TSlot)] 
          if (Employee, Day, getPreviousTimeslot(TSlot), Assignment) in m.works)
    
    return obj

In [91]:
# Add Objective function to the model
model.obj = Objective(rule=obj_rule, sense=minimize)

### Constraints

In [92]:
# Model constraints
model.constraints = ConstraintList()  # Create a Set of Constraints

#### 1. Employee Allocation <= Requirement

In [93]:
# Maximum Employee Requirement
for Day in Days:
    for TSlot in TSlots:
        for Assignment in Ass_Tslots[(Day,TSlot)]:
            model.constraints.add(
                Emp_Req[(Day,TSlot)][Assignment] >= sum(model.works[Employee, Day, TSlot, Assignment] for Employee in Employees)
            )

#### 2. Daily Max work hours

In [94]:
# Max Work Hours
for Employee in Employees:
    model.constraints.add(
        EmpMaxWork[(Day, Employee)] >= 0.25 * sum(
            model.works[Employee, Day, TSlot, Assignment] for Day in Days for TSlot in TSlots for Assignment in Ass_Tslots[(Day, TSlot)])
    )

#### 3. Daily Min work hours

In [95]:
# Min Work Hours
for Employee in Employees:
    model.constraints.add(
        EmpMinWork[(Day,Employee)] <= 0.25 * sum(
            model.works[Employee, Day, TSlot, Assignment] for Day in Days for TSlot in TSlots for Assignment in Ass_Tslots[(Day,TSlot)]) + model.MI
    )

#### 4. No overlapping assignments for employees

In [96]:
# No allocation for overlapping Assignments
for Employee in Employees:
    for Day in Days:
        for TSlot in TSlots:
            model.constraints.add(
                1 >= sum(model.works[Employee, Day, TSlot, Assignment] for Assignment in Ass_Tslots[(Day,TSlot)])
            )

#### 5. Employee eligibility to task

In [97]:
# Employee Eligibility to Allocate to Assignments (remove this with decision variable)
for Employee in Employees:
    for (Day,TSlot) in Ass_Tslots:
        for Assignment in list(set(Ass_Tslots[(Day,TSlot)]).intersection(set(Assignments) - set(Emp_Elig[(Day,Employee)]))):
            model.constraints.add(
                0 == model.works[Employee, Day, TSlot, Assignment]
            )

#### 6. Weekly Min work hours

In [98]:
#Weekly Min Work Hours
for Employee in Employees:
    model.constraints.add(
        EmpWeeklyMinHours[Employee] <= 0.25 * sum(
            model.works[Employee, Day, TSlot, Assignment] for Day in Days for TSlot in TSlots for Assignment in Ass_Tslots[(Day, TSlot)]) + model.MIW
    )

#### 7. Weekly Max work hours

In [99]:
EmpWeeklyMinHours

{'Employee501603': 8,
 'Employee501715': 8,
 'Employee501716': 8,
 'Employee501721': 8,
 'Employee501768': 12,
 'Employee501792': 12,
 'Employee501587': 12,
 'Employee501595': 12}

In [100]:
# Weekly Max Work Hours
for Employee in Employees:
    model.constraints.add(
        EmpWeeklyMaxHours[Employee] >= 0.25 * sum(
            model.works[Employee, Day, TSlot, Assignment] for Day in Days for TSlot in TSlots for Assignment in Assignments if (Employee, Day, TSlot, Assignment) in model.works ) 
    )

#### 8. Weekly Max days

In [101]:
# for Employee in Employees:
#     model.constraints.add(
#         EmpWeeklyMaxDays[Employee] >= sum(
#             model.dayWorks[Employee, Day] for Day in Days) 
#     )

#### Link employee assignment for timeslot with employee assignment for day

In [102]:
for Employee in Employees:
    for Day in Days:
        model.constraints.add(
            0.25 * sum(model.works[Employee, Day, TSlot, Assignment] for TSlot in TSlots for Assignment in Ass_Tslots[(Day, TSlot)]) <= EmpMaxWork[(Day, Employee)] * model.dayWorks[Employee, Day]
        )

In [103]:
for Employee in Employees:
    model.constraints.add(
        EmpWeeklyMaxDays[Employee] >= sum(
            model.dayWorks[Employee, Day] for Day in Days) 
    )

#### 9. Pinned and must assign tasks for employees

In [104]:
PinnedTaskList

[('Employee501603', '15/11/2018', datetime.time(9, 0), 'Downtown Platform'),
 ('Employee501603', '15/11/2018', datetime.time(9, 15), 'Downtown Platform'),
 ('Employee501603', '15/11/2018', datetime.time(9, 30), 'Downtown Platform'),
 ('Employee501603', '15/11/2018', datetime.time(9, 45), 'Downtown Platform'),
 ('Employee501715', '16/11/2018', datetime.time(9, 0), 'Downtown Lobby'),
 ('Employee501715', '16/11/2018', datetime.time(9, 15), 'Downtown Lobby'),
 ('Employee501715', '16/11/2018', datetime.time(9, 30), 'Downtown Lobby'),
 ('Employee501715', '16/11/2018', datetime.time(9, 45), 'Downtown Lobby')]

In [105]:
for (Employee, Day, TSlot, Assignment) in PinnedTaskList:
    model.constraints.add(
        1 == model.works[Employee, Day, TSlot, Assignment] 
    )

In [106]:
# Constraint - An employee can't be allocated to one task more than once within the day
# for Employee in Employees:
#     for Assignment in Assignments: 
#         model.constraints.add(
#             1 >= sum(model.works[Employee, TSlot, Assignment] - model.works[Employee, getPreviousTimeslot(TSlot), Assignment] 
#               for TSlot in TSlots[1:] if ((Employee,getPreviousTimeslot(TSlot), Assignment) in model.works and (Employee,TSlot, Assignment) in model.works ))
#         )


In [107]:
# Constraint - An employee must work in one section atleast for an hour
#TODO: Should be able to take the minimum hours to work in a section as an input



In [108]:
opt = SolverFactory('cbc')
results = opt.solve(model, tee=True)
log_infeasible_constraints(model)  #Get infeasible constraints

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Aug 21 2017 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpsgnepdox.pyomo.lp -stat=1 -solve -solu /tmp/tmpsgnepdox.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 248 (-2233) rows, 680 (-1579) columns and 2625 (-14672) elements
Statistics for presolved model
Original problem has 2256 integers (2256 of which binary)
Presolved problem has 680 integers (680 of which binary)
==== 8 zero objective 2 different
672 variables have objective of -3
8 variables have objective of 0
==== absolute objective values 2 different
8 variables have objective of 0
672 variables have objective of 3
==== for integers 8 zero objective 2 different
672 variables have objective of -3
8 variables have objective of 0
==== for integers absolute objective values 2 different
8 variables have objective of 0
672 variables have objective of 3
===== end objective counts


Problem has 248 rows, 68

INFO: CONSTR constraints[764]: 0.0 </= 1.0
INFO: CONSTR constraints[766]: 0.0 </= 1.0
INFO: CONSTR constraints[768]: 0.0 </= 1.0
INFO: CONSTR constraints[769]: 0.0 </= 1.0
INFO: CONSTR constraints[771]: 0.0 </= 1.0
INFO: CONSTR constraints[773]: 0.0 </= 1.0
INFO: CONSTR constraints[774]: 0.0 </= 1.0
INFO: CONSTR constraints[775]: 0.0 </= 1.0
INFO: CONSTR constraints[776]: 0.0 </= 1.0
INFO: CONSTR constraints[783]: 0.0 </= 1.0
INFO: CONSTR constraints[792]: 0.0 </= 1.0
INFO: CONSTR constraints[797]: 0.0 </= 1.0
INFO: CONSTR constraints[798]: 0.0 </= 1.0
INFO: CONSTR constraints[799]: 0.0 </= 1.0
INFO: CONSTR constraints[800]: 0.0 </= 1.0
INFO: CONSTR constraints[801]: 0.0 </= 1.0
INFO: CONSTR constraints[802]: 0.0 </= 1.0
INFO: CONSTR constraints[803]: 0.0 </= 1.0
INFO: CONSTR constraints[809]: 0.0 </= 1.0
INFO: CONSTR constraints[810]: 0.0 </= 1.0
INFO: CONSTR constraints[812]: 0.0 </= 1.0
INFO: CONSTR constraints[813]: 0.0 </= 1.0
INFO: CONSTR constraints[814]: 0.0 </= 1.0
INFO: CONST

In [127]:
model.pprint()

3 Set Declarations
    constraints_index : Dim=0, Dimen=1, Size=2480, Domain=None, Ordered=False, Bounds=None
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198,

                 ('Employee501768', '16/11/2018', datetime.time(3, 0), 'Downtown Lobby') :     0 :   1.0 :     1 : False : False : Binary
              ('Employee501768', '16/11/2018', datetime.time(3, 0), 'Downtown Platform') :     0 :   0.0 :     1 : False : False : Binary
                ('Employee501768', '16/11/2018', datetime.time(3, 15), 'ABM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                 ('Employee501768', '16/11/2018', datetime.time(3, 15), 'BM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                ('Employee501768', '16/11/2018', datetime.time(3, 15), 'Downtown Lobby') :     0 :   1.0 :     1 : False : False : Binary
             ('Employee501768', '16/11/2018', datetime.time(3, 15), 'Downtown Platform') :     0 :   0.0 :     1 : False : False : Binary
                ('Employee501768', '16/11/2018', datetime.time(3, 30), 'ABM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                 ('Employee501768'

        Key  : Active : Sense    : Expression
        None :   True : minimize : 3*(649.1 - (works[Employee501603,15/11/2018,09:00:00,Downtown Lobby] + works[Employee501603,15/11/2018,09:00:00,Downtown Platform] + works[Employee501603,15/11/2018,09:00:00,BM Admin Time] + works[Employee501603,15/11/2018,09:00:00,ABM Admin Time] + works[Employee501603,15/11/2018,09:15:00,Downtown Lobby] + works[Employee501603,15/11/2018,09:15:00,Downtown Platform] + works[Employee501603,15/11/2018,09:15:00,BM Admin Time] + works[Employee501603,15/11/2018,09:15:00,ABM Admin Time] + works[Employee501603,15/11/2018,09:30:00,Downtown Lobby] + works[Employee501603,15/11/2018,09:30:00,Downtown Platform] + works[Employee501603,15/11/2018,09:30:00,BM Admin Time] + works[Employee501603,15/11/2018,09:30:00,ABM Admin Time] + works[Employee501603,15/11/2018,09:45:00,Downtown Lobby] + works[Employee501603,15/11/2018,09:45:00,Downtown Platform] + works[Employee501603,15/11/2018,09:45:00,BM Admin Time] + works[Employee

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [110]:
# Print Employees who assigned to each assignment in all Time Slots

Dic = {Day: {TSlot: {Assignment: [] for Assignment in Ass_Tslots[(Day,TSlot)]} for TSlot in TSlots} for Day in Days}
for Employee in Employees:
    for Day in Days:
        for TSlot in TSlots:
            for Assignment in Ass_Tslots[(Day,TSlot)]:
                if model.works[Employee, Day, TSlot, Assignment] == 1:
                    Dic[Day][TSlot][Assignment].append(Employee)

### Day1 Schedule

In [111]:
Day1 = Dic['15/11/2018']

In [112]:
#print(pd.DataFrame(Day1))

In [113]:
day1Df =pd.DataFrame(Day1)
day1Df.head(20)

,09:00:00,09:15:00,09:30:00,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,...,03:30:00,03:45:00,04:00:00,04:15:00,04:30:00,04:45:00,08:30:00,08:45:00,05:00:00,05:15:00
Downtown Lobby,[],[],[],[],[Employee501587],[],[],[],"[Employee501768, Employee501587]",[Employee501587],...,[],[Employee501587],[],[Employee501587],[],[],NaN,NaN,NaN,NaN
Downtown Platform,[Employee501603],[Employee501603],[Employee501603],"[Employee501603, Employee501595]","[Employee501603, Employee501595]","[Employee501603, Employee501792, Employee501595]","[Employee501792, Employee501595]","[Employee501603, Employee501595]","[Employee501603, Employee501792]","[Employee501603, Employee501792]",...,"[Employee501603, Employee501792, Employee501595]","[Employee501603, Employee501792]","[Employee501603, Employee501792, Employee501595]","[Employee501603, Employee501595]",[Employee501603],"[Employee501603, Employee501792]",NaN,NaN,NaN,NaN
BM Admin Time,[],[],[],[],[],[],[],[],[],[],...,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABM Admin Time,[],[],[],[],[],[],[],[],[],[],...,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BMT Meeting,NaN,NaN,NaN,NaN,[],[],[],[],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Teller Opener Platform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],NaN,NaN
MSR Opener Platform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[Employee501595],[Employee501603],NaN,NaN
Teller Opener Lobby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],NaN,NaN
MSR Opener Lobby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],NaN,NaN
Teller Closer Platform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]


In [114]:
day1Df.iloc[1]

09:00:00                                    [Employee501603]
09:15:00                                    [Employee501603]
09:30:00                                    [Employee501603]
09:45:00                    [Employee501603, Employee501595]
10:00:00                    [Employee501603, Employee501595]
10:15:00    [Employee501603, Employee501792, Employee501595]
10:30:00                    [Employee501792, Employee501595]
10:45:00                    [Employee501603, Employee501595]
11:00:00                    [Employee501603, Employee501792]
11:15:00                    [Employee501603, Employee501792]
11:30:00                    [Employee501603, Employee501595]
11:45:00                                    [Employee501603]
12:00:00                    [Employee501603, Employee501792]
12:15:00                    [Employee501603, Employee501792]
12:30:00                    [Employee501603, Employee501595]
12:45:00                    [Employee501603, Employee501792]
01:00:00                

In [115]:
Day2 = Dic['16/11/2018']

In [116]:
day2Df =pd.DataFrame(Day2)
day2Df.head(20)

,09:00:00,09:15:00,09:30:00,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,...,03:30:00,03:45:00,04:00:00,04:15:00,04:30:00,04:45:00,08:30:00,08:45:00,05:00:00,05:15:00
Downtown Lobby,"[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716]","[Employee501715, Employee501716, Employee50176...",[Employee501715],[Employee501716],"[Employee501715, Employee501716, Employee50176...",[Employee501716],"[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee501768]","[Employee501715, Employee501716, Employee501768]",...,"[Employee501716, Employee501768, Employee501587]","[Employee501715, Employee501716, Employee501768]","[Employee501715, Employee501716, Employee501768]","[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee50176...",NaN,NaN,NaN,NaN
Downtown Platform,[],"[Employee501792, Employee501595]","[Employee501792, Employee501595]",[Employee501721],"[Employee501792, Employee501595]",[],[Employee501595],[Employee501595],"[Employee501792, Employee501595]",[Employee501792],...,[Employee501792],"[Employee501792, Employee501595]","[Employee501721, Employee501792, Employee50158...","[Employee501721, Employee501792, Employee501595]","[Employee501721, Employee501792]","[Employee501721, Employee501595]",NaN,NaN,NaN,NaN
BM Admin Time,[Employee501721],[Employee501721],[Employee501721],[],[Employee501721],[Employee501721],[],[Employee501721],[],[Employee501721],...,[Employee501721],[Employee501721],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABM Admin Time,[],[],[],[],[],[],[],[],[],[],...,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BMT Meeting,NaN,NaN,NaN,NaN,[],[],[],[],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Teller Opener Platform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],NaN,NaN
MSR Opener Platform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[Employee501721],[],NaN,NaN
Teller Opener Lobby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],NaN,NaN
MSR Opener Lobby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[Employee501715],[Employee501715],NaN,NaN
Teller Closer Platform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]


In [117]:
day2Df.iloc[0]

09:00:00    [Employee501715, Employee501716, Employee50176...
09:15:00                     [Employee501715, Employee501716]
09:30:00    [Employee501715, Employee501716, Employee50176...
09:45:00                                     [Employee501715]
10:00:00                                     [Employee501716]
10:15:00    [Employee501715, Employee501716, Employee50176...
10:30:00                                     [Employee501716]
10:45:00    [Employee501715, Employee501716, Employee50176...
11:00:00     [Employee501715, Employee501716, Employee501768]
11:15:00     [Employee501715, Employee501716, Employee501768]
11:30:00     [Employee501715, Employee501716, Employee501768]
11:45:00     [Employee501715, Employee501716, Employee501768]
12:00:00    [Employee501715, Employee501716, Employee50176...
12:15:00    [Employee501715, Employee501716, Employee50176...
12:30:00    [Employee501715, Employee501716, Employee50176...
12:45:00    [Employee501715, Employee501716, Employee50176...
01:00:00

### Output Final Schedule

In [118]:
#Employee Daily hours
EmpAss = {}
for Employee in Employees:    
    for Day in Days:
        hours = 0
        for TSlot in TSlots:
            for Assignment in Ass_Tslots[(Day, TSlot)]:
                if model.works[Employee, Day, TSlot, Assignment] == 1:
                    hours = hours + 0.25
        EmpAss[(Day,Employee)] = hours
EmpAss


{('15/11/2018', 'Employee501603'): 8.0,
 ('16/11/2018', 'Employee501603'): 0,
 ('15/11/2018', 'Employee501715'): 0,
 ('16/11/2018', 'Employee501715'): 8.0,
 ('15/11/2018', 'Employee501716'): 0,
 ('16/11/2018', 'Employee501716'): 8.0,
 ('15/11/2018', 'Employee501721'): 0,
 ('16/11/2018', 'Employee501721'): 8.0,
 ('15/11/2018', 'Employee501768'): 1.0,
 ('16/11/2018', 'Employee501768'): 7.0,
 ('15/11/2018', 'Employee501792'): 3.75,
 ('16/11/2018', 'Employee501792'): 4.25,
 ('15/11/2018', 'Employee501587'): 3.5,
 ('16/11/2018', 'Employee501587'): 5.5,
 ('15/11/2018', 'Employee501595'): 3.5,
 ('16/11/2018', 'Employee501595'): 4.5}

In [119]:
#Employee Weekly hours
EmpAss2 = {}
for Employee in Employees:
    hours = 0
    for Day in Days:
        for TSlot in TSlots:
            for Assignment in Ass_Tslots[(Day, TSlot)]:
                if model.works[Employee, Day, TSlot, Assignment] == 1:
                    hours = hours + 0.25
    EmpAss2[Employee] = hours
EmpAss2


{'Employee501603': 8.0,
 'Employee501715': 8.0,
 'Employee501716': 8.0,
 'Employee501721': 8.0,
 'Employee501768': 8.0,
 'Employee501792': 8.0,
 'Employee501587': 9.0,
 'Employee501595': 8.0}